In [ ]:
import pickle
import pandas as pd
import numpy as np
import math
import random
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
with open('song_id.txt', 'rb') as handle:
    unserialized_song_id = pickle.load(handle)

with open('user_song.dict', 'rb') as handle:
    unserialized_user_song = pickle.load(handle)
    
with open('user_song_tuple.txt', 'rb') as handle:
    unserialized_user_song_tuple = pickle.load(handle)

In [ ]:
user_song_train = {}
user_song_test = {}
user_song_tuple_train_pos = []
user_song_tuple_train_neg = []
user_song_tuple_test_pos = []
user_song_tuple_test_neg = []

for user in tqdm(unserialized_user_song):
    # pos
    pos_songs = unserialized_user_song[user]
    random.shuffle(pos_songs)
    cut = int(len(pos_songs)*0.8)
    user_song_train[user] = pos_songs[:cut]
    for song in pos_songs[:cut]:
        user_song_tuple_train_pos.append([user, song])
    user_song_test[user] = songs[cut:]
    for song in pos_songs[cut:]:
        user_song_tuple_test_pos.append([user, song])
    # neg
    pos_song_set = set(pos_songs)
    random.shuffle(unserialized_song_id)
    j = 0
    for i in range(len(user_song_train[user])*4):
        while j < len(unserialized_song_id) and unserialized_song_id[j] in pos_song_set:
            j += 1
        if j >= len(unserialized_song_id):
            break
        user_song_tuple_train_neg.append([user, unserialized_song_id[i]])
        j += 1
    for i in range(len(user_song_test[user])*100):
        while j < len(unserialized_song_id) and unserialized_song_id[j] in pos_song_set:
            j += 1
        if j >= len(unserialized_song_id):
            break
        user_song_tuple_test_neg.append([user, unserialized_song_id[i]])
        j += 1

In [ ]:
with open('user_song_train.dict', 'wb') as handle:
    pickle.dump(user_song_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_test.dict', 'wb') as handle:
    pickle.dump(user_song_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_train_pos.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_train_neg.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_pos.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_neg.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)